<a href="https://colab.research.google.com/github/dmeoli/NeuroSAT/blob/master/AlphaZeroSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [ ]:
import os

if not os.path.isdir('neuroSAT'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuroSAT
  %cd neuroSAT
else:
  %cd neuroSAT
  !git pull

/content/gdrive/My Drive/neuro-sat
Already up to date.


In [ ]:
!pip install -r requirements.txt

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 124 kB 7.2 MB/s 
     |████████████████████████████████| 308 kB 44.9 MB/s 
     |████████████████████████████████| 2.9 MB 6.7 MB/s 
     |████████████████████████████████| 8.0 MB 20.2 MB/s 
     |████████████████████████████████| 376 kB 56.7 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.0-py3-none-any.whl size=513273 sha256=70e4b8ca32c9726fc3172adf84c627bebc2b4f34b439041b58dcec0cd149c84a
  Stored in directory: /root/.cache/pip/wheels/b5/22/3f/10c97635d8d73e012582116b4e449d47fc64b7246cc71085fc
Successfully built torch-geometric


In [ ]:
datasets = {'uniform-random-3-sat': {'train': ['uf50-218', 'uuf50-218',
                                               'uf100-430', 'uuf100-430'],
                                     'val': ['uf50-218', 'uuf50-218',
                                             'uf100-430', 'uuf100-430'],
                                     'inner_test': ['uf50-218', 'uuf50-218',
                                                    'uf100-430', 'uuf100-430'],
                                     'test': ['uf250-1065', 'uuf250-1065']},
            'graph-coloring': {'train': ['flat50-115'],
                               'val': ['flat50-115'],
                               'inner_test': ['flat50-115'],
                               'test': ['flat30-60',
                                        'flat75-180',
                                        'flat100-239',
                                        'flat125-301',
                                        'flat150-360',
                                        'flat175-417',
                                        'flat200-479']}}

In [ ]:
%cd AlphaZeroSAT

/content/gdrive/My Drive/neuro-sat/AlphaZeroSAT


## Build C++

In [ ]:
%cd MCTSminisat
!apt install libgsl-dev
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuro-sat/AlphaZeroSAT/MCTSminisat
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgsl23 libgslcblas0
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html
The following NEW packages will be installed:
  libgsl-dev libgsl23 libgslcblas0
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 1,926 kB of archives.
After this operation, 9,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgslcblas0 amd64 2.4+dfsg-6 [79.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl23 amd64 2.4+dfsg-6 [823 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl-dev amd64 2.4+dfsg-6 [1,023 kB]
Fetched 1,926 kB in 11s (183 kB/s)
Selecting previously unselected package libgslcblas0:amd64.
(Reading database ... 148492 files and directo

## Uniform Random 3-SAT

We split *(u)uf50-218* and *(u)uf100-430* into three subsets: 800 training problems, 100 validation, and 100 test problems.

For generalization experiments, we use 100 problems from all the other benchmarks.

To evaluate the knowledge transfer properties of the trained models across different task families, we use 100 problems from all the *graph coloring* benchmarks.

In [ ]:
PROBLEM_TYPE='uniform-random-3-sat'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, TEST_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['inner_test']):
  !python train.py \
    --save_dir runs/"$PROBLEM_TYPE"/"$TRAIN_PROBLEM_NAME" \
    --best_model -1 \
    --status_file status.pkl \
    --result_file result.pkl \
    --dump_dir parameters \
    --dump_file sl.pkl \
    --train_path ../data/"$PROBLEM_TYPE"/"$TRAIN_PROBLEM_NAME" \
    --test_path ../data/"$PROBLEM_TYPE"/"$TEST_PROBLEM_NAME" \
    --max_clause 120 \
    --max_var 20 \
    --sl_buffer_size 1000000 \
    --n_batch 32 \
    --n_stack 1 \
    --n_repeat 100 \
    --n_start 0 \
    --n_train_files 0 \
    --n_test_files 0 \
    --l2_coeff 0.0001 \
    --sl_num_steps 64000 \
    --sl_n_batch 32 \
    --sl_n_checkpoint 32000 \
    --n_cycles 2 \
    --show_only 'No' \
    --which_cycle 0

## Graph Coloring

Graph coloring benchmarks have only 100 problems each, except for *flat50-115* which contains 1000, so we split it into three subsets: 800 training problems, 100 validation, and 100 test problems.

For generalization experiments, we use 100 problems from all the other benchmarks.

In [ ]:
PROBLEM_TYPE='graph-coloring'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, TEST_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['inner_test']):
  !python train.py \
    --save_dir runs/"$PROBLEM_TYPE"/"$TRAIN_PROBLEM_NAME" \
    --best_model -1 \
    --status_file status.pkl \
    --result_file result.pkl \
    --dump_dir parameters \
    --dump_file sl.pkl \
    --train_path ../data/"$PROBLEM_TYPE"/"$TRAIN_PROBLEM_NAME" \
    --test_path ../data/"$PROBLEM_TYPE"/"$TEST_PROBLEM_NAME" \
    --max_clause 120 \
    --max_var 20 \
    --sl_buffer_size 1000000 \
    --n_batch 32 \
    --n_stack 1 \
    --n_repeat 100 \
    --n_start 0 \
    --n_train_files 0 \
    --n_test_files 0 \
    --l2_coeff 0.0001 \
    --sl_num_steps 64000 \
    --sl_n_batch 32 \
    --sl_n_checkpoint 32000 \
    --n_cycles 2 \
    --show_only 'No' \
    --which_cycle 0